In [2]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import os
from pandastable import Table, TableModel

class_ion_data_path = '/Users/nedasmatulionis/Documents/Programming/christofk-lab/hek_std_model/mets_w_classes_hek_model.xlsx'

# Scoring contribution of variables
peak_area_impact = 0.4  # Positive impact
iqr_diff_impact = 0.1  # Negative impact
variability_impact = 0.2  # Negative impact
outlier_impact = 0.3  # Negative impact

# Color thresholds


class MetaboliteAnalysisApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Metabolite Standard Peak Area Analysis")
        self.geometry("1200x1600")
        
        self.notebook = ttk.Notebook(self)
        self.notebook.pack(fill='both', expand=True)

        # Load stored data at initialization
        self.stored_data = self.load_stored_data()

        # Load class and ion data
        self.class_ion_data = self.load_class_ion_data(class_ion_data_path)
        self.create_class_ion_mappings()

        self.setup_met_std_peak_area_check_ui()

    def load_class_ion_data(self, filepath):
        class_ion_data = pd.read_excel(filepath)
        class_ion_data.columns = class_ion_data.columns.str.strip()
        class_ion_data['metabolite'] = class_ion_data['metabolite'].str.replace('|', ';').str.replace('"', '').str.strip()
        class_ion_data['class'] = class_ion_data['class'].str.replace('"', '').str.strip()
        class_ion_data['ion'] = class_ion_data['ion'].str.replace('"', '').str.strip()
        class_ion_data.set_index('metabolite', inplace=True)
        return class_ion_data

    def create_class_ion_mappings(self):
        self.class_mapping = self.class_ion_data['class'].to_dict()
        self.ion_mapping = self.class_ion_data['ion'].to_dict()

    def clean_dataframe_index(self, df):
        df.index = df.index.str.replace('|', ';').str.replace('"', '').str.strip()
        return df

    def setup_met_std_peak_area_check_ui(self):
        self.tab_peak_area_check = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_peak_area_check, text='Std Peak Area Check')

        self.file_path_frame_peak_area = tk.Frame(self.tab_peak_area_check)
        self.file_path_frame_peak_area.pack(fill='x', pady=15)

        self.file_path_entry_peak_area = tk.Entry(self.file_path_frame_peak_area)
        self.file_path_entry_peak_area.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.upload_button_peak_area = tk.Button(self.file_path_frame_peak_area, text="Upload Met Std Peak Area Data", command=self.upload_std_peak_area_file)
        self.upload_button_peak_area.pack(side=tk.RIGHT, padx=5)

        self.std_met_peak_area_stats = tk.Frame(self.tab_peak_area_check)
        self.std_met_peak_area_stats.pack(fill='x', pady=15)

        self.met_stats_table = ttk.Treeview(self.std_met_peak_area_stats,
                                            columns=(
                                                'Compound',
                                                'Mean',
                                                'min-max',
                                                'RSD',
                                                'IQR',
                                                'N Mean',
                                                'N min-max',
                                                'N RSD',
                                                'N IQR',
                                                'Score',
                                                'N Score'),
                                            show="headings", height=20)
        self.met_stats_table.heading("Compound", text="Compound")
        self.met_stats_table.column("Compound", width=170, anchor=tk.CENTER)
        self.met_stats_table.heading("Mean", text="Mean", anchor=tk.CENTER)
        self.met_stats_table.column("Mean", width=70, anchor=tk.CENTER)
        self.met_stats_table.heading("min-max", text="min-max", anchor=tk.CENTER)
        self.met_stats_table.column("min-max", width=120, anchor=tk.CENTER)
        self.met_stats_table.heading("RSD", text="RSD (%)", anchor=tk.CENTER)
        self.met_stats_table.column("RSD", width=30, anchor=tk.CENTER)
        self.met_stats_table.heading("IQR", text="IQR", anchor=tk.CENTER)
        self.met_stats_table.column("IQR", width=70, anchor=tk.CENTER)
        self.met_stats_table.heading("N Mean", text="N Mean", anchor=tk.CENTER)
        self.met_stats_table.column("N Mean", width=70, anchor=tk.CENTER)
        self.met_stats_table.heading("N min-max", text="N min-max", anchor=tk.CENTER)
        self.met_stats_table.column("N min-max", width=120, anchor=tk.CENTER)
        self.met_stats_table.heading("N RSD", text="N RSD (%)", anchor=tk.CENTER)
        self.met_stats_table.column("N RSD", width=30, anchor=tk.CENTER)
        self.met_stats_table.heading("N IQR", text="N IQR", anchor=tk.CENTER)
        self.met_stats_table.column("N IQR", width=70, anchor=tk.CENTER)
        self.met_stats_table.heading("Score", text="Score", anchor=tk.CENTER)
        self.met_stats_table.column("Score", width=60, anchor=tk.CENTER)
        self.met_stats_table.heading("N Score", text="N Score", anchor=tk.CENTER)
        self.met_stats_table.column("N Score", width=60, anchor=tk.CENTER)
        self.met_stats_table.pack(fill='x', pady=5, padx=15)

        self.met_stats_table.tag_configure('green', background='lightgreen')
        self.met_stats_table.tag_configure('yellow', background='yellow')
        self.met_stats_table.tag_configure('red', background='lightcoral')

        self.met_stats_table.bind("<Double-1>", self.on_double_click)

        # Adding buttons for sorting
        self.button_frame = tk.Frame(self.tab_peak_area_check)
        self.button_frame.pack(fill='x', pady=10)

        self.sort_by_class_button = tk.Button(self.button_frame, text="Sort by Class", command=self.sort_by_class)
        self.sort_by_class_button.pack(side=tk.LEFT, padx=5)

        self.sort_by_ion_button = tk.Button(self.button_frame, text="Sort by Ionization", command=self.sort_by_ion)
        self.sort_by_ion_button.pack(side=tk.LEFT, padx=5)

        self.display_info_button = tk.Button(self.button_frame, text="Class and Ion Info", command=self.display_metabolite_info)
        self.display_info_button.pack(side=tk.LEFT, padx=5)

        self.class_groups = {}
        self.ion_groups = {}

    def get_color_tag(self, score):
        if score > 0.5:
            return 'green'
        elif score < 0:
            return 'red'
        else:
            return 'yellow'




    def sort_by_class(self):
        self.update_treeview(sort_by='class')

    def sort_by_ion(self):
        self.update_treeview(sort_by='ion')


    def display_metabolite_info(self):
        def sort_column(tree, col, reverse):
            data_list = [(tree.set(item, col), item) for item in tree.get_children('')]
            data_list.sort(reverse=reverse)
            for index, (_, item) in enumerate(data_list):
                tree.move(item, '', index)
            tree.heading(col, command=lambda: sort_column(tree, col, not reverse))

        def search_metabolites(event):
            query = search_var.get().lower()
            for item in tree.get_children():
                tree.delete(item)
            for metabolite, row in self.class_ion_data.iterrows():
                if query in metabolite.lower() or query in row['class'].lower() or query in row['ion'].lower():
                    tree.insert('', tk.END, values=(metabolite, row['class'], row['ion']))

        info_window = tk.Toplevel(self)
        info_window.title("Metabolite Class and Ionization Info")
        info_window.geometry("600x400")

        search_var = tk.StringVar()
        search_bar = tk.Entry(info_window, textvariable=search_var)
        search_bar.pack(fill='x')
        search_bar.bind('<KeyRelease>', search_metabolites)

        tree = ttk.Treeview(info_window, columns=('Metabolite', 'Class', 'Ion'), show='headings')
        tree.heading('Metabolite', text='Metabolite', command=lambda: sort_column(tree, 'Metabolite', False))
        tree.heading('Class', text='Class', command=lambda: sort_column(tree, 'Class', False))
        tree.heading('Ion', text='Ion', command=lambda: sort_column(tree, 'Ion', False))

        tree.column('Metabolite', width=200, anchor=tk.W)
        tree.column('Class', width=200, anchor=tk.W)
        tree.column('Ion', width=100, anchor=tk.W)

        for metabolite, row in self.class_ion_data.iterrows():
            tree.insert('', tk.END, values=(metabolite, row['class'], row['ion']))

        tree.pack(fill='both', expand=True)

        scrollbar = ttk.Scrollbar(info_window, orient="vertical", command=tree.yview)
        tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side='right', fill='y')








    def upload_std_peak_area_file(self):
        analysis_fpath = filedialog.askopenfilename()
        if not analysis_fpath:
            return

        self.file_path_entry_peak_area.delete(0, tk.END)
        self.file_path_entry_peak_area.insert(0, analysis_fpath)

        if self.is_excel_file(analysis_fpath):
            xls = pd.ExcelFile(analysis_fpath)
            if 'PoolAfterDF' in xls.sheet_names:
                self.df = pd.read_excel(analysis_fpath, sheet_name='PoolAfterDF', index_col='Compound')
                self.df = self.clean_dataframe_index(self.df)  # Clean the DataFrame index
                self.df_normalized = self.df.div(self.df.loc['trifluoromethanesulfonate'])
                self.update_treeview()
            else:
                messagebox.showerror("Error", "'PoolAfterDF' sheet not present in the Excel file.")
        else:
            messagebox.showerror("Error", "Selected file is not a valid Excel file.")


    def is_excel_file(self, fpath):
        return fpath.endswith(('.xls', '.xlsx'))
    
    def load_stored_data(self):
        directory = os.path.expanduser('~/Documents/Programming/christofk-lab/hek_std_model/hek_model_stored')
        stored_data = {}
        if not os.path.exists(directory):
            print(f"Directory not found: {directory}")
            return {}  # Return an empty dictionary if the directory does not exist
        for file in os.listdir(directory):
            if file.endswith('.xlsx'):
                path = os.path.join(directory, file)
                try:
                    df_original = pd.read_excel(path, index_col='Compound')
                    if 'trifluoromethanesulfonate' in df_original.index:
                        df_normalized = df_original.div(df_original.loc['trifluoromethanesulfonate'])
                        date_label = file.split('_')[0]
                        stored_data[date_label] = (df_original, df_normalized)
                except Exception as e:
                    print(f"Failed to load {file}: {e}")
        return stored_data
    
    def calculate_rsd(self, data):
        """Calculate the Relative Standard Deviation (RSD), handling cases where the mean is zero."""
        mean = data.mean()
        if mean == 0:
            return 0  # Return 0 or some other appropriate value instead of NaN
        return (data.std() / mean) * 100
        
    def calculate_iqr_and_range(self, data):
        """Calculates the interquartile range and min-max of the given data."""
        q75, q25 = np.percentile(data, [75 ,25])
        iqr = q75 - q25
        data_min = np.min(data)
        data_max = np.max(data)
        min_max_range = f"{data_min:.2e}-{data_max:.2e}"
        return iqr, min_max_range
    
    def calculate_scores(self):
        # Calculate scores for original and normalized data
        original_scores = self.calculate_data_scores(self.df, normalized=False)
        normalized_scores = self.calculate_data_scores(self.df_normalized, normalized=True)

        # Normalize the scores
        # original_scores = self.normalize_scores(original_scores)
        # normalized_scores = self.normalize_scores(normalized_scores)

        # Combine original and normalized scores into a single dictionary
        scores = {compound: (original_scores.get(compound, 'ND'), normalized_scores.get(compound, 'ND'))
                for compound in self.df.index}
        return scores


    def calculate_data_scores(self, df, normalized):
        stored_means = {}
        stored_iqrs = {}
        internal_standard = 'trifluoromethanesulfonate'

        for date_label, (df_orig, df_norm) in self.stored_data.items():
            df_to_use = df_norm if normalized else df_orig
            for compound in df_to_use.index:
                if compound not in stored_means:
                    stored_means[compound] = []
                    stored_iqrs[compound] = []
                values = df_to_use.loc[compound].values.flatten()
                stored_means[compound].extend(values)
                q75, q25 = np.percentile(values, [75, 25])
                iqr = q75 - q25
                stored_iqrs[compound].append(iqr)

        # Calculate mean and IQR of the stored data
        stored_means = {compound: np.mean(values) for compound, values in stored_means.items()}
        stored_iqrs = {compound: np.mean(values) for compound, values in stored_iqrs.items()}

        scores = {}
        for compound in df.index:
            if compound in stored_means:
                values = df.loc[compound].values.flatten()
                mean_peak_area = np.mean(values)
                iqr_diff = abs(np.percentile(values, 75) - np.percentile(values, 25) - stored_iqrs[compound])

                # Calculate relative measures
                mean_peak_area_score = mean_peak_area / stored_means[compound] if stored_means[compound] != 0 else mean_peak_area
                iqr_diff_score = iqr_diff / stored_iqrs[compound] if stored_iqrs[compound] != 0 else iqr_diff

                # Internal standard adjustment for normalized data
                if normalized and internal_standard in df.index:
                    internal_standard_ratio = df.loc[internal_standard].mean() / self.stored_data_mean(internal_standard, normalized)
                    mean_peak_area_score *= internal_standard_ratio

                # Calculate variability impact
                variability_impact = self.calculate_variability_impact(values)

                # Detect and penalize outliers
                outlier_impact = self.detect_outliers(values)




                # Combine the scores with weights: mean positively, IQR difference negatively, variability negatively, outliers negatively
                peak_area_contrib = mean_peak_area_score * peak_area_impact  # Positive impact
                iqr_diff_contrib = -iqr_diff_score * iqr_diff_impact  # Negative impact
                variability_contrib = -variability_impact * variability_impact  # Negative impact
                outlier_contrib = -outlier_impact * outlier_impact  # Negative impact

                scores[compound] = peak_area_contrib + iqr_diff_contrib + variability_contrib + outlier_contrib
        return scores

    def stored_data_mean(self, compound, normalized):
        means = []
        for date_label, (df_orig, df_norm) in self.stored_data.items():
            df_to_use = df_norm if normalized else df_orig
            if compound in df_to_use.index:
                means.append(df_to_use.loc[compound].mean())
        return np.mean(means) if means else 1  # Return 1 if no means found to avoid division by zero

    def calculate_variability_impact(self, data):
        # Calculate the standard deviation as a measure of variability
        return data.std() / data.mean() if data.mean() != 0 else data.std()

    def detect_outliers(self, data):
        # Calculate the number of outliers based on IQR
        q75, q25 = np.percentile(data, [75, 25])
        iqr = q75 - q25
        lower_bound = q25 - 1.5 * iqr
        upper_bound = q75 + 1.5 * iqr
        outliers = [x for x in data if x < lower_bound or x > upper_bound]
        outlier_impact = len(outliers) / len(data)  # Proportion of outliers
        return outlier_impact

    def normalize_scores(self, scores):
        min_score = min(scores.values())
        max_score = max(scores.values())
        range_score = max_score - min_score
        if range_score == 0:
            return {compound: 0.5 for compound in scores}
        return {compound: (score - min_score) / range_score for compound, score in scores.items()}

    def update_treeview(self, sort_by=None):
        scores = self.calculate_scores()  # Calculate scores if scoring is applied

        # Clear existing entries in the TreeView
        for i in self.met_stats_table.get_children():
            self.met_stats_table.delete(i)

        if sort_by == 'class':
            sorted_compounds = sorted(self.df.index, key=lambda x: self.class_mapping.get(x.replace('|', ';').replace('"', '').strip(), ''))
        elif sort_by == 'ion':
            sorted_compounds = sorted(self.df.index, key=lambda x: self.ion_mapping.get(x.replace('|', ';').replace('"', '').strip(), ''))
        else:
            sorted_compounds = self.df.index

        # Special handling to ensure trifluoromethanesulfonate is added first
        special_compound = 'trifluoromethanesulfonate'
        if special_compound in sorted_compounds:
            self.add_compound_to_treeview(special_compound, scores)
            sorted_compounds = [comp for comp in sorted_compounds if comp != special_compound]

        # Inserting all other compounds into the TreeView
        for compound in sorted_compounds:
            self.add_compound_to_treeview(compound, scores)

    def get_color_for_score(self, score):
        if score > 0.5:
            return "lightgreen"
        elif score < 0:
            return "lightcoral"
        else:
            return "yellow"


    def add_compound_to_treeview(self, compound, scores):
        original_data = self.df.loc[compound]
        normalized_data = self.df_normalized.loc[compound]

        # Calculate RSD
        original_rsd = self.calculate_rsd(original_data)
        normalized_rsd = self.calculate_rsd(normalized_data)

        # Calculate IQR and min-max for original and normalized data
        original_iqr, original_min_max = self.calculate_iqr_and_range(original_data)
        normalized_iqr, normalized_min_max = self.calculate_iqr_and_range(normalized_data)

        score = scores[compound][0] if scores and compound in scores else "N/A"
        n_score = scores[compound][1] if scores and compound in scores else "N/A"

        # Prepare data for insertion
        values = (
            compound.replace('"', ''),
            f"{original_data.mean():.3e}",  # Mean
            original_min_max,  # Min-Max
            f"{original_rsd:.2f}",  # RSD
            f"{original_iqr:.3e}",  # IQR
            f"{normalized_data.mean():.3e}",  # Norm Mean
            normalized_min_max,  # Norm Min-Max
            f"{normalized_rsd:.2f}",  # Norm RSD
            f"{normalized_iqr:.3e}",  # Norm IQR
            f"{score:.3f}" if score != "N/A" else "N/A",  # Score for original data
            f"{n_score:.3f}" if n_score != "N/A" else "N/A"   # N Score for normalized data
        )

        item_id = self.met_stats_table.insert("", "end", values=values)

        # Apply color to the Score column
        if score != "N/A":
            self.met_stats_table.set(item_id, column="Score", value=f"{score:.3f}", tags=(self.get_color_tag(float(score)),))

        # Apply color to the N Score column
        if n_score != "N/A":
            self.met_stats_table.set(item_id, column="N Score", value=f"{n_score:.3f}", tags=(self.get_color_tag(float(n_score)),))


    def on_double_click(self, event):
        # Identify the item and column clicked on
        item_id = self.met_stats_table.identify_row(event.y)
        column_id = self.met_stats_table.identify_column(event.x)
        
        # Get the name of the compound from the row
        compound = self.met_stats_table.item(item_id, 'values')[0]
        
        # Decide which dataset to use based on the column clicked
        # '#2' is 'Mean', '#3' is min-max, '#6' is 'N Mean', '#7' is min-max
        if column_id in ["#2", "#3"]:  
            data = self.df.loc[compound]
            title_suffix = "Original Replicates"
            self.plot_data_points_scatter(data, compound, title_suffix)

        elif column_id in ["#6", "#7"]:
            data = self.df_normalized.loc[compound]
            title_suffix = "Normalized Replicates"
            self.plot_data_points_scatter(data, compound, title_suffix)

        elif column_id in ["#4", "#5"]:  # These are the columns for 'RSD', 'IQR'
            data = self.df.loc[compound]
            title_suffix = "Original"
            self.plot_data_points(data, compound, title_suffix)

        elif column_id in ["#8", "#9"]:  # These are the columns for 'N RSD', 'N IQR'
            data = self.df_normalized.loc[compound]
            title_suffix = "Normalized"
            self.plot_data_points(data, compound, title_suffix)

        elif column_id in ["#10", "#11"]:  # These are the columns for 'Score' and 'N Score'
            if column_id == "#10":
                title_suffix = "Original with Stored Data"
                normalized = False
            else:
                title_suffix = "Normalized with Stored Data"
                normalized = True
            self.plot_box_w_stored(compound, title_suffix, normalized)

    def plot_box_w_stored(self, compound_name, title_suffix, normalized):
        if not self.stored_data:
            messagebox.showerror("Error", "Stored data is not available.")
            return

        data_lists = []
        labels = []

        # Prepare data from stored files
        stored_data_entries = []
        for date_label, (df_orig, df_norm) in self.stored_data.items():
            df_to_use = df_norm if normalized else df_orig
            if compound_name in df_to_use.index:
                data = df_to_use.loc[compound_name].values.flatten()
                stored_data_entries.append((date_label, data))

        # Sort data by date label to ensure consistent order
        stored_data_entries.sort(key=lambda x: x[0])

        # Append sorted data
        for date_label, data in stored_data_entries:
            data_lists.append(data)
            labels.append(date_label)

        # Add the new data last
        if compound_name in (self.df_normalized if normalized else self.df).index:
            current_data = (self.df_normalized if normalized else self.df).loc[compound_name].values.flatten()
            data_lists.append(current_data)
            labels.append('New')  # This label will appear last

        # Ensure all entries in data_lists are 1D arrays
        data_lists = [np.array(data).flatten() for data in data_lists]

        # Generate plot if there is any data to show
        if data_lists:
            popup = tk.Toplevel()
            popup.title(f"{title_suffix} - {compound_name}")
            popup.geometry("1000x600")

            fig, ax = plt.subplots()
            ax.boxplot(data_lists, tick_labels=labels, notch=True, patch_artist=True)
            ax.set_title(f"{title_suffix} - {compound_name}")
            ax.set_ylabel('Normalized Peak Area Values' if normalized else 'Original Peak Area Values')
            ax.set_xlabel('Date')
            ax.tick_params(axis='x', rotation=45)

            plt.tight_layout()
            canvas = FigureCanvasTkAgg(fig, master=popup)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
            tk.Button(popup, text="Close", command=popup.destroy).pack(side=tk.BOTTOM)
        else:
            messagebox.showinfo("Data Unavailable", f"No data available for {compound_name}.")


    def plot_data_points_scatter(self, data, compound_name, title_suffix):
        # Create a popup window for the scatter plot
        popup = tk.Toplevel()
        popup.title(f"{compound_name} - {title_suffix}")
        popup.geometry("1000x1000")
        
        fig, ax = plt.subplots()
        # Plot data points
        ax.scatter(range(len(data)), data, color='blue', alpha=0.7, label=f'{compound_name} data')
        
        # Customizing the plot
        ax.set_title(f"{compound_name} - {title_suffix}")
        ax.set_ylabel("Values")
        ax.set_xlabel("Sample Index")
        plt.xticks(rotation=45)

        # Creating a canvas as a matplotlib backend
        canvas = FigureCanvasTkAgg(fig, master=popup)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        
        # Add a close button to the popup
        tk.Button(popup, text="Close", command=popup.destroy).pack(side=tk.BOTTOM)

    def plot_data_points(self, data, compound_name, title_suffix):
        # Prepare data by grouping by the initial part of the column name
        groups = {}
        for col in data.dropna().index:
            date = col.split('-')[0]  # Assuming date is the first part before '-HEK-std'
            if date not in groups:
                groups[date] = []
            groups[date].append(data[col])

        # Setup the popup window
        popup = tk.Toplevel()
        popup.title(f"{compound_name} - {title_suffix}")
        popup.geometry("1000x1000")

        fig, ax = plt.subplots()
        # Create boxplot for each group of data
        box_data = [groups[date] for date in sorted(groups)]
        bp = ax.boxplot(box_data, tick_labels=sorted(groups.keys()), notch=True, vert=True, patch_artist=True, showfliers=True)

        # Customize the boxplot appearance
        for box in bp['boxes']:
            # Set edge color and fill with a more transparent color
            box.set(color='#1f77b4', linewidth=2)
            box.set(facecolor='#1f77b4', alpha=0.5)  # Set transparency

        for whisker in bp['whiskers']:
            whisker.set(color='#1f77b4', linewidth=2)

        for cap in bp['caps']:
            cap.set(color='#1f77b4', linewidth=2)

        for median in bp['medians']:
            median.set(color='yellow', linewidth=2)  # Set medians to yellow for visibility

        for flier in bp['fliers']:
            flier.set(marker='o', color='#e7298a', alpha=0.9)  # Outliers visible as pink dots

        # Add individual data points on the plot for clarity
        for i, line in enumerate(groups):
            y_data = groups[line]
            x_data = np.random.normal(1 + i, 0.02, size=len(y_data))  # Add some jitter to the x-axis
            ax.plot(x_data, y_data, 'r.', alpha=0.5)  # Points are plotted as red dots with transparency

        ax.set_title(f"{compound_name} - {title_suffix}")
        ax.set_ylabel("Peak Area")
        ax.set_xlabel("Date Run")
        plt.xticks(rotation=45)
        plt.grid(True)

        # Create a canvas as a matplotlib backend
        canvas = FigureCanvasTkAgg(fig, master=popup)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Add a close button to the popup
        tk.Button(popup, text="Close", command=popup.destroy).pack(side=tk.BOTTOM)

if __name__ == "__main__":
    app = MetaboliteAnalysisApp()
    app.mainloop()


wrote config file /Users/nedasmatulionis/.config/pandastable/default.conf


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tkinter/__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/g7/8z1_10qj4276f7r_8x84bd_40000gn/T/ipykernel_31495/4082250304.py", line 110, in upload_std_peak_area_file
    self.update_table()
  File "/var/folders/g7/8z1_10qj4276f7r_8x84bd_40000gn/T/ipykernel_31495/4082250304.py", line 268, in update_table
    self.met_stats_table.show()
  File "/Users/nedasmatulionis/Documents/Programming/.lab/lib/python3.12/site-packages/pandastable/core.py", line 306, in show
    self.toolbar = ToolBar(self.parentframe, self)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nedasmatulionis/Documents/Programming/.lab/lib/python3.12/site-packages/pandastable/core.py", line 3719, in __init__
    addButton(self, 'Load table', self.parentapp.load, img, 'load table')
  File "/Users/nedasmatuli

KeyboardInterrupt: 